In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler, LabelEncoder

2025-04-03 15:16:50.902972: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-03 15:16:51.738935: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743693411.933686    1840 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743693411.983440    1840 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743693412.428442    1840 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
train = pd.read_csv("../DATA/train.csv")
train = train.dropna()


/tmp/ipykernel_1840/1474832752.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("../DATA/train.csv")


In [3]:
train['Protocol'] = pd.to_numeric(train['Protocol'], errors='coerce')
train = train.dropna(subset=['Protocol'])
train['Protocol'] = train['Protocol'].astype(int)

In [4]:
x = train.drop('Label', axis=1)
y = train['Label']

In [5]:
for col in x.columns:
    x[col] = pd.to_numeric(x[col], errors='coerce')
x = x.fillna(0)

In [6]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
le = LabelEncoder()
y_encoded = le.fit_transform(y)
num_classes = len(le.classes_)

In [10]:
input_layer = Input(shape=(x_scaled.shape[1],))

# Embedding layer
x = Dense(128, activation='gelu')(input_layer)

# Positional Encoding as a Keras Layer
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def build(self, input_shape):
        position = np.arange(input_shape[1])[:, np.newaxis]
        angle_rates = 1 / np.power(10000, (2 * (np.arange(input_shape[2]) // 2) / np.float32(input_shape[2])))
        angle_rads = position * angle_rates[np.newaxis, :]
        
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
        self.pos_encoding = tf.constant(angle_rads[np.newaxis, ...], dtype=tf.float32)
        
    def call(self, inputs):
        return inputs + self.pos_encoding

x = PositionalEncoding()(x)


IndexError: tuple index out of range

In [8]:
attn_output = MultiHeadAttention(num_heads=4, key_dim=32)(x, x)
x = LayerNormalization(epsilon=1e-6)(x + attn_output)
x = Dropout(0.1)(x)
ffn_output = Dense(512, activation='gelu')(x)
ffn_output = Dense(128)(ffn_output)
x = LayerNormalization(epsilon=1e-6)(x + ffn_output)

# Classification Head
x = tf.reduce_mean(x, axis=1)
output = Dense(num_classes, activation='softmax')(x)

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.ops`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
